# Terrestrial vs Oceanic Water Storage and ENSO
Ruth Moorman May 7th 2024 (for Carmen Blackwood ESE144 lectures)

In this notebook we'll be looking into the relationship between El Nino/La Nina and terrestrial water storage. In particular, we'll be reproducing some of the analysis from 
<a href="https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2012GL053055">"The 2011 La Nina: So strong the oceans fell" (2012) by Carmen and colleagues</a>.


<div class="alert alert-block alert-info">
<b>DATA:</b>
<div> 
<div>        
We'll again be using data from JPL's <b>Physical Oceanography Distributed Active Archive Center (PODAAC)</b>.
<div><div>

Here, in particular, we'll be using <a href="https://podaac.jpl.nasa.gov/dataset/TELLUS_GRAC-GRFO_MASCON_CRI_GRID_RL06.1_V3#">JPL GRACE and GRACE-FO Mascon Ocean, Ice, and Hydrology Equivalent Water Height Coastal Resolution Improvement (CRI) Filtered Release 06.1 Version 03</a>, which is the latest, near-real time release of JPL GRACE "mascon" (mass concentration) gravity anomaly solutions.

I also used <a href="https://psl.noaa.gov/data/timeseries/monthly/NINO34/">this Nino3.4 timeseries</a>.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings('ignore')


In [ ]:
## load data 
jpl_mascons = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/GRCTellus.JPL.200204_202402.GLO.RL06.1M.MSCNv03CRI.nc')
jpl_mascons

## Timeseries of terrestrial and ocean mass changes

We want to make a timeseries of the LWE thickness anomaly over the land and over the oceans. 

To do that comparison, we'll need to keep in mind that the mean LWE thickness anomaly over the oceans will not be directly comparable to the mean LWE thickness anomaly over land, since these values represent masses over water spread over different areas. Instead we'll compare LWE volume anomalies by first multiplying LWE Thickness anomalies by cell area. 

The data product does not come with a cell area coordinate so I have crudely (assuming the Earth is a sphere, which it isn't) computed one from the latitude and longitude grid (this may introduce some errors if it doesn't match what the GRACE JPL team used in processing). I calculate the cell area according to,

$$ A = R^2 (\sin{\phi_1}-\sin{\phi_2})(\theta_1 - \theta_2)$$

where $R$ is the Earth's radius (approximate), $\phi$ terms are the latitude bounds of the grid, and $\theta$ terms are the longitude bounds (both in radians).



In [ ]:
land_mask = jpl_mascons.land_mask

In [ ]:
## creat a cell area array (units m)
lat_bounds = jpl_mascons.lat_bounds.values
lon_bounds = jpl_mascons.lon_bounds.values
R = 6378 * 1e3
sinϕ1_minus_sinϕ2   = np.sin(np.deg2rad(lat_bounds[:,0])) - np.sin(np.deg2rad(lat_bounds[:,1])) # one dimesional array
_,sinϕ1_minus_sinϕ2 = np.meshgrid(np.zeros(720),sinϕ1_minus_sinϕ2) # make into a grid
θ1_minus_θ2         = np.deg2rad(lon_bounds[0,0]) - np.deg2rad(lon_bounds[0,1]) ## this is constant for this grid
cell_area           = R**2 * sinϕ1_minus_sinϕ2 * θ1_minus_θ2
cell_area           = xr.DataArray(cell_area, coords = land_mask.coords)

In [ ]:
lwe_volume = jpl_mascons.lwe_thickness * 100 * cell_area # thickness is in cm, here is m3

In [ ]:
## mask JPL mascons using land mask
ocean_mascons = lwe_volume.where(land_mask==0)
land_mascons  = lwe_volume.where(land_mask==1)

In [ ]:
## sum spatially
land_mascons_sum    = land_mascons.sum({'lon', 'lat'})
ocean_mascons_sum   = ocean_mascons.sum({'lon', 'lat'})


<div class="alert alert-block alert-info">
<b>NOTE:</b>

When we're interested in long term trends or interannual variations of climate quantities, it's common to remove a monthly "climatology". This means we remove the mean January value from each January datapoint, the mean February value from each February datapoint, and so on.
<div>
Using xarray this is quatire simply with the <a href ="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html">groupby function</a>.

In [ ]:
## remove climatology
land_mascons_sum   = (land_mascons_sum.groupby('time.month')  - land_mascons_sum.groupby('time.month').mean()).load() # remove monthly climatology
ocean_mascons_sum  = (ocean_mascons_sum.groupby('time.month') - ocean_mascons_sum.groupby('time.month').mean()).load() # remove monthly climatology



<div class="alert alert-block alert-info">
<b>NOTE:</b>

We'll use another xarray function, <a href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.rolling.html">rolling</a>, to compute running means.<div>


In [ ]:
## running mean
land_mascons_sum_running  = land_mascons_sum.rolling(time=3, center=True).mean().load()
ocean_mascons_sum_running = ocean_mascons_sum.rolling(time=3, center=True).mean().load()

In [ ]:
nino34 = np.loadtxt('nino34_timeseries.txt', usecols=np.arange(1,13)).flatten()
nino34 = xr.DataArray(nino34, coords = [np.arange('1950-01', '2025-01', dtype='datetime64[M]').astype('datetime64[ns]')+np.timedelta64(14,'D')], dims='time')
nino34 = nino34.sel(time=slice('2002-04','2024-02'))

In [ ]:
fig,(ax,ax1) = plt.subplots(2,1, figsize=(12,9),gridspec_kw={'hspace':0.1, 'height_ratios':[1,0.3]})
ax.plot(ocean_mascons_sum.time, ocean_mascons_sum, 'bo-', alpha=0.2,markersize=2,label='Monthly GRACE over ocean')
ax.plot(ocean_mascons_sum_running.time, ocean_mascons_sum_running, 'b-', linewidth=2, label='3 month running mean')
ax.plot(land_mascons_sum.time, -land_mascons_sum, 'ko-', alpha=0.2,markersize=2,label='-(Monthly GRACE over land)')
ax.plot(land_mascons_sum_running.time, -land_mascons_sum_running, 'k-', linewidth=2, label='-(3 month running mean)')
ax.legend(loc=0)


## Nino 3.4 plots
ax1.plot(nino34.time, nino34, 'k-')
ax1.fill_between(nino34.time, nino34.where(nino34>=0.4),0.4,color='r', alpha=0.2)
ax1.fill_between(nino34.time, nino34.where(nino34<=-0.4),-0.4,color='b', alpha=0.2)
ax1.axhline(0,color='k', linewidth=0.5)
ax1.axhline(0.4,color='k',linestyle='--', linewidth=0.5)
ax1.axhline(-0.4,color='k', linestyle='--',linewidth=0.5)
ax1.set_ylim([-2,3])

ax.grid(axis='x', linestyle=':')
ax1.grid(axis='x', linestyle=':')
ax.set_ylabel('Integrated LWE Volume Anomaly (m$^3$)')
ax1.set_ylabel('Nino 3.4 Index')
plt.show()

## Mapping the terrestrial water storage anomaly between 2010 and 2011 (Fig 3) - La Nina signal

Fig 3 in Boening et al 2011 shows the anomaly between the 2010 JFM (January-February-March) and 2011 MAM (March-April-May) mean terrestrial water storage. This is replicated here. 


In [ ]:
## mask JPL mascons using land mask, we'll use the land masked values here
land_mascons  = jpl_mascons.lwe_thickness.where(land_mask==1)

In [ ]:
JFM_2010    = land_mascons.sel(time=slice('2010-01','2010-03')).mean('time')
MAM_2011    = land_mascons.sel(time=slice('2011-03','2011-05')).mean('time')
TWS_anomaly = (MAM_2011 - JFM_2010)*10 # convert to mm

In [ ]:
fig = plt.figure(figsize=[15, 7])
proj = ccrs.Robinson(central_longitude=160)
ax = plt.subplot(projection = proj)
cf=ax.pcolormesh(TWS_anomaly.lon, TWS_anomaly.lat, TWS_anomaly, transform = ccrs.PlateCarree(), vmin=-150,vmax=150, cmap='RdBu')
ax.contour(land_mask.lon, land_mask.lat,land_mask, levels=[0,1], colors=['k'],transform = ccrs.PlateCarree())

cax = fig.add_axes([0.26, 0.05, 0.5, 0.03])
cbar=plt.colorbar(cf,cax = cax,orientation='horizontal',shrink = 0.5)
cax.set_xlabel('Equivalent Water Height [mm]', fontsize = 15)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,linewidth=1, color='gray', alpha=0.5, linestyle='--')
ax.set_title('MAM 2011 - JFM 2010 Terrestrial Water Storage', fontsize=15)
plt.show()




<div class="alert alert-block alert-info">
<b>DISCUSS:</b>

How does this compare to expected <a href="https://sotp.nyc3.cdn.digitaloceanspaces.com/wp-content/uploads/2016/04/elnino-lanina-teleconnections.jpg">La Nina rainfall patterns</a>?
 <div><div>
     


   
The JPL GRACE solutions appear more 'blocky' or low resolution than the solutions presented in Boening et al (2011), is this a true difference in resolution? Discuss some of the differences between GRACE solutions released by different analysis centers.


## Mapping the terrestrial water storage anomaly between 2015 and 2016 (New) - El Nino signal

Here let's compare 2015 JFM (January-February-March, start of El Nino) and 2016 MAM (March-April-May, end of El Nino) mean terrestrial water storage.


In [ ]:
JFM_2015    = land_mascons.sel(time=slice('2015-01','2015-03')).mean('time')
MAM_2016    = land_mascons.sel(time=slice('2016-03','2016-05')).mean('time')
TWS_anomaly = (MAM_2016 - JFM_2015)*10 # convert to mm

In [ ]:
fig = plt.figure(figsize=[15, 7])
proj = ccrs.Robinson(central_longitude=160)
ax = plt.subplot(projection = proj)
cf=ax.pcolormesh(TWS_anomaly.lon, TWS_anomaly.lat, TWS_anomaly, transform = ccrs.PlateCarree(), vmin=-150,vmax=150, cmap='RdBu')
ax.contour(land_mask.lon, land_mask.lat,land_mask, levels=[0,1], colors=['k'],transform = ccrs.PlateCarree())

cax = fig.add_axes([0.26, 0.05, 0.5, 0.03])
cbar=plt.colorbar(cf,cax = cax,orientation='horizontal',shrink = 0.5)
cax.set_xlabel('Equivalent Water Height [mm]', fontsize = 15)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,linewidth=1, color='gray', alpha=0.5, linestyle='--')
ax.set_title('MAM 2015 - JFM 2016 Terrestrial Water Storage', fontsize=15)
plt.show()




<div class="alert alert-block alert-info">
<b>DISCUSS:</b>

How does this compare to expected El Nino rainfall patterns?
    
    

## How much of Terrestrial Water Storage interannual variability is explained by El Nino?

(a preliminary look...)


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div>
    
 Reproduce the time series figure from above but with <b>detrended</b> GRACE Land and Ocean storage.
   <div>
 
Do this by computing linear fits to the datasets (e.g. using <a href="https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html">np.polyfit</a>) and removing that linear function from the data.

   <div>


In [ ]:
## compute linear fits... ##

x = ocean_mascons_sum.time.astype('float') # can't use time dimension in datetime64 format
y_ocean = ocean_mascons_sum
y_land  = land_mascons_sum

## your code here... ##

# fit_ocean = 
# fit_land = 

## remove linear fit from data
# detrended_ocean = 
# detrended_land  = 

## running mean
# detrended_ocean_running = 
# detrended_land_running  = 


In [ ]:
fig,(ax,ax1) = plt.subplots(2,1, figsize=(15,9),gridspec_kw={'hspace':0.1, 'height_ratios':[1,1]})

## your code here...(uncomment and alter) ##
# ax.plot(,, 'bo-', alpha=0.2,markersize=2,label='Monthly GRACE over ocean [detrended]')
# ax.plot(,, 'b-', linewidth=2, label='3 month running mean [detrended]')
# ax.plot(,, 'ko-', alpha=0.2,markersize=2,label='-(Monthly GRACE over land) [detrended]')
# ax.plot(,, 'k-', linewidth=2, label='-(3 month running mean) [detrended]')
# ax.legend(loc=0)
# ax.axhline(0,color='k', linewidth=0.5)


## Nino 3.4 plots (unchanged)
ax1.plot(nino34.time, nino34, 'k-')
ax1.fill_between(nino34.time, nino34.where(nino34>=0.4),0.4,color='r', alpha=0.2)
ax1.fill_between(nino34.time, nino34.where(nino34<=-0.4),-0.4,color='b', alpha=0.2)
ax1.axhline(0,color='k', linewidth=0.5)
ax1.axhline(0.4,color='k',linestyle='--', linewidth=0.5)
ax1.axhline(-0.4,color='k', linestyle='--',linewidth=0.5)
ax1.set_ylim([-2,3])

ax.grid(axis='x', linestyle=':')
ax1.grid(axis='x', linestyle=':')
ax.set_ylabel('Integrated LWE Volume Anomaly (m$^3$)')
ax1.set_ylabel('Nino 3.4 Index')
plt.show()


<div class="alert alert-block alert-info">


Discuss the relationship between El Nino and TWS.